# Simulation of an Electric System based on Thermoelectrics Plants

In [14]:
from src.map import Map2D, GraphMap
from src.utils.gaussianmixture import DailyElectricityConsumptionBimodal
from src.circuits import Circuit, Block
from src.thermoelectrics import Thermoelectric
from src.part import Coils, Boiler, Generator, SteamTurbine
import pandas as pd
import plotly.express as px
from numpy import random

from src.simulation_constants import (
    NO_CIRCUITS,
    NO_THERMOELECTRICS,
    MIN_CITIZEN,
    MAX_CITIZEN,
    MAX_DEVIATION_CITIZEN_IN_BLOCK,
    DEMAND_PER_PERSON,
    DEMAND_INDUSTRIALIZATION,
    VARIABILITY_DEMAND_PER_PERSON,
    VARIABILITY_DEMAND_PER_INDUSTRIALIZATION,
    PEAK_CONSUMPTION_MORNING,
    PEAK_CONSUMPTION_EVENING,
    MAX_DEVIATION_MORNING,
    MAX_DEVIATION_EVENING,
    WEIGHT_MORNING,
    WEIGHT_EVENING,
    MIN_BLOCKS_PER_CIRCUIT,
    MAX_BLOCKS_PER_CIRCUIT,
    IMPORTANCE_ALPHA,
    RANDOM_SEED,
    DISTANCE_REGULATOR
)

## Initialize bases 

In [15]:
map_2d = Map2D(
    no_circuits=NO_CIRCUITS,
    no_thermoelectrics=NO_THERMOELECTRICS,
)


map_2d.visualize()

In [16]:
graphMap = GraphMap(
    thermoelectric_labels=[f"Th{i}" for i in range(NO_THERMOELECTRICS)],
    circuits_labels=[f"C{i}" for i in range(NO_CIRCUITS)],
    towers_labels=[f"Tw{i}" for i in range(len(map_2d.towers_positions))],
    thermoelectrics_positions=map_2d.thermoelectrics_positions,
    circuits_positions=map_2d.circuits_positions,
    towers_positions=map_2d.towers_positions,
)

graphMap.visualize()

In [28]:
distance_cost_template = graphMap.thermoelectric_generation_cost

# Generate circuits for map
random.seed(RANDOM_SEED)
RANDOM = random.default_rng(RANDOM_SEED)


electric_system_circuits_list: list[Circuit] = []
for i in range(NO_CIRCUITS):
    citizen_count = RANDOM.integers(MIN_CITIZEN, MAX_CITIZEN)
    citizen_range = (
        max(citizen_count - MAX_DEVIATION_CITIZEN_IN_BLOCK, 0),
        min(citizen_count + MAX_DEVIATION_CITIZEN_IN_BLOCK, MAX_CITIZEN),
    )

    industrialization = RANDOM.integers(0, DEMAND_INDUSTRIALIZATION) / DEMAND_INDUSTRIALIZATION
    
    print(citizen_count)

    print(industrialization)

    bimodal_consumption = DailyElectricityConsumptionBimodal(
        base_consumption=DEMAND_PER_PERSON * citizen_count
        + DEMAND_INDUSTRIALIZATION * industrialization,
        base_variability=VARIABILITY_DEMAND_PER_PERSON * citizen_count
        + VARIABILITY_DEMAND_PER_INDUSTRIALIZATION * industrialization,
        mean_morning=PEAK_CONSUMPTION_MORNING,
        mean_evening=PEAK_CONSUMPTION_EVENING,
        std_morning=RANDOM.uniform(1.0, MAX_DEVIATION_MORNING),
        std_evening=RANDOM.uniform(1.0, MAX_DEVIATION_EVENING),
        weight_morning=WEIGHT_MORNING,
        weight_evening=WEIGHT_EVENING,
    )

    electric_system_circuits_list.append(
        Circuit(
            graphMap.circuits_nodes[i].id,
            gaussian_mixture=bimodal_consumption,
            blocks_range=(MIN_BLOCKS_PER_CIRCUIT, MAX_BLOCKS_PER_CIRCUIT),
            citizens_range=citizen_range,
            industrialization=industrialization,
        )
    )

# Generate Thermoelectric generation based in the nearest circuits
mapper_circuit_with_thermoelectric = {}

for c in electric_system_circuits_list:

    filtered = [f for f in graphMap.thermoelectric_generation_cost if f[1] == c.id]
    filtered = sorted(filtered, key=lambda x: x[2])
    mapper_circuit_with_thermoelectric[c.id] = filtered[0][0]


import plotly.graph_objects as go

# Create edges for the plotly graph
edges = []
edges_labels = []
for circuit, thermoelectric in mapper_circuit_with_thermoelectric.items():

    circuitPos = [c.position for c in graphMap.circuits_nodes if c.id == circuit][0]
    thermoelectricPos = [
        t.position for t in graphMap.thermoelectrics_nodes if t.id == thermoelectric
    ][0]

    edges.append((circuitPos, thermoelectricPos))
    edges_labels.append(f"{circuit} -> {thermoelectric}")


# Create a plotly graph
fig = go.Figure()

# Add edges to the plotly graph
for i, edge in enumerate(edges):
    circuit, thermoelectric = edge
    fig.add_trace(
        go.Scatter(
            x=[circuit[0], thermoelectric[0]],
            y=[circuit[1], thermoelectric[1]],
            mode="lines",
            line=dict(color="black"),
            name="Wire Connection",
            showlegend=False,
            hoverinfo="text",
            text=edges_labels[i],
            hoverlabel=dict(bgcolor="white", font_size=16, font_family="Rockwell"),
        )
    )

# Add nodes to the plotly graph
all_nodes = graphMap.circuits_nodes + graphMap.thermoelectrics_nodes
for i, node in enumerate(all_nodes):
    fig.add_trace(
        go.Scatter(
            x=[node.position[0]],
            y=[node.position[1]],
            mode="markers+text",
            marker=dict(
                size=10, color="blue" if i >= len(graphMap.circuits_nodes) else "red"
            ),
            text=[node.id],
            textposition="top center",
            hoverinfo="text",
        )
    )

fig.update_layout(
    showlegend=False,
    title="Circuits and Thermoelectrics",
    height=800,
    plot_bgcolor="lightgreen",
)

fig.show()

4542
0.8
4744
0.2
3256
0.6
1182
0.9
4563
0.0
3625
0.4
4604
0.8
1830
0.7
2339
0.7
4780
0.5
3625
0.2
4585
0.5
1074
0.7
1169
0.9
2690
0.7
4734
0.6
4858
0.5
3727
0.8
4730
0.5
3764
0.5
3602
0.4
1537
0.0
1538
0.5
2792
0.1
1560
0.4
4534
0.0
3120
0.8
3817
0.3
1015
0.8
4374
0.9
4782
0.7
2436
0.6
4134
0.7
695
0.4
2649
0.3
3043
0.3
3851
0.6
2412
0.7
2812
0.7
3257
0.3
3082
0.4
4589
0.2
4796
0.9
691
0.5
4765
0.0
2391
0.0
2336
0.9
1898
0.4
3332
0.2
1215
0.9


In [18]:
# Generate the distance matrix


def distance_template_to_distance_matrix(
    template: list[tuple[str, str, float, list[str]]],
    thermoelectrics: list[str],
    circuits: list[str],
):
    matrix = [[-1 for _ in range(len(circuits))] for _ in range(len(thermoelectrics))]

    c_map = {}
    t_map = {}

    for i, t in enumerate(thermoelectrics):
        t_map[t] = i

    for i, c in enumerate(circuits):
        c_map[c] = i

    max_cost = 1
    for t, c, cost, _ in template:
        matrix[t_map[t]][c_map[c]] = cost
        max_cost = max(max_cost, cost)

    for t, c, _, _ in template:
        matrix[t_map[t]][c_map[c]] /= max_cost * DISTANCE_REGULATOR

    return matrix


matrix = distance_template_to_distance_matrix(
    distance_cost_template,
    [t.id for t in graphMap.thermoelectrics_nodes],
    [c.id for c in graphMap.circuits_nodes],
)


distance_matrix_df = pd.DataFrame(
    matrix,
    index=[t.id for t in graphMap.thermoelectrics_nodes],
    columns=[c.id for c in graphMap.circuits_nodes],
)

distance_matrix_df

,C0,C1,C2,C3,C4,C5,C6,C7,C8,C9,...,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49
Th0,0.002410,0.003950,0.001736,0.006778,0.005999,0.002119,0.002534,0.000945,0.002204,0.001529,...,0.007186,0.001896,0.002305,0.002224,0.005232,0.001272,0.000126,0.005332,0.001703,0.000165
Th1,0.009244,0.003239,0.005175,0.000547,0.001358,0.008954,0.009368,0.006265,0.009038,0.005480,...,0.000955,0.008730,0.009139,0.009058,0.001629,0.008106,0.006961,0.001773,0.008538,0.006999
Th2,0.003338,0.003327,0.001112,0.006155,0.005375,0.003048,0.003462,0.000359,0.003132,0.000906,...,0.006563,0.002824,0.003233,0.003152,0.004608,0.002200,0.001055,0.004708,0.002632,0.001093
Th3,0.007654,0.001649,0.003585,0.001629,0.000850,0.007364,0.007778,0.004675,0.007448,0.003890,...,0.002037,0.007140,0.007549,0.007468,0.000083,0.006516,0.005371,0.000183,0.006948,0.005409
Th4,0.000781,0.005623,0.003408,0.008450,0.007671,0.000490,0.000904,0.002618,0.000575,0.003202,...,0.008858,0.000266,0.000676,0.000594,0.006904,0.000793,0.001799,0.007004,0.000092,0.001603
Th5,0.008358,0.002353,0.004289,0.000888,0.000472,0.008068,0.008482,0.005379,0.008152,0.004594,...,0.001296,0.007844,0.008253,0.008172,0.000743,0.007220,0.006075,0.000887,0.007652,0.006113
Th6,0.001379,0.005071,0.002856,0.007899,0.007119,0.001088,0.001503,0.002066,0.001173,0.002650,...,0.008307,0.000864,0.001274,0.001193,0.006352,0.000241,0.001247,0.006452,0.000672,0.001051
Th7,0.000274,0.006086,0.003872,0.008914,0.008135,0.000256,0.000398,0.003081,0.000068,0.003665,...,0.009322,0.000356,0.000169,0.000088,0.007368,0.001256,0.002262,0.007468,0.000556,0.002067


In [19]:
# Generate thermoelectrics
electric_system_thermoelectrics_list: list[Thermoelectric] = []

for i, t in enumerate(graphMap.thermoelectrics_nodes):
    generated_thermoelectric_min_cost = 0

    for j, c in enumerate(graphMap.circuits_nodes):
        if mapper_circuit_with_thermoelectric[c.id] == t.id:
            generated_thermoelectric_min_cost += (
                electric_system_circuits_list[j].mock_electric_consume
                + electric_system_circuits_list[j].mock_electric_consume
                * matrix[i][j]
                * 24
            )

    electric_system_thermoelectrics_list.append(
        Thermoelectric(
            id=t.id,
            total_capacity=generated_thermoelectric_min_cost + 200,
        )
    )
    thermoelectric_data = {
        "ID": [t.id for t in electric_system_thermoelectrics_list],
        "Total Capacity": [
            t.total_capacity for t in electric_system_thermoelectrics_list
        ],
    }

    thermoelectric_df = pd.DataFrame(thermoelectric_data)


sorted_electric_system_circuit_list = sorted(
    electric_system_circuits_list,
    key=lambda x: list(
        filter(
            lambda y: mapper_circuit_with_thermoelectric[x.id] == y.id,
            graphMap.thermoelectrics_nodes,
        )
    )[0].id,
)

colors = [
    list(
        filter(
            lambda x, y=circuit: mapper_circuit_with_thermoelectric[y.id] == x.id,
            graphMap.thermoelectrics_nodes,
        )
    )[0].id
    for circuit in sorted_electric_system_circuit_list
]


fig = px.bar(
    thermoelectric_df,
    x="ID",
    y="Total Capacity",
    title="Total Capacity of Thermoelectrics",
    height=800,
    color=[t.id for t in electric_system_thermoelectrics_list],
    color_discrete_sequence=px.colors.qualitative.Plotly,
    labels={"color": "Thermoelectric ID"},
)

fig.show()

In [20]:

mock_electric_consume_data = {
    "Circuit ID": [c.id for c in sorted_electric_system_circuit_list],
    "Mock Electric Consume": [c.mock_electric_consume for c in sorted_electric_system_circuit_list],
}

mock_electric_consume_df = pd.DataFrame(mock_electric_consume_data)

fig = px.bar(
    mock_electric_consume_df,
    x="Circuit ID",
    y="Mock Electric Consume",
    title="Electric Consume per Circuit [24 hours]",
    height=800,
    color=colors
)
fig.show()

In [21]:
max_population_of_circuits = -1
max_population_of_block = -1

for circuit in electric_system_circuits_list:
    max_population_of_circuits = max(
        circuit.get_all_block_population(), max_population_of_circuits
    )
    for block in circuit.blocks:
        max_population_of_block = max(block.citizens.amount, max_population_of_block)

auxiliary_data_max_population_of_circuits = max_population_of_circuits
auxiliary_data_max_population_of_block = max_population_of_block


def get_circuit_importance(circuit: Circuit) -> float:
    return (
        circuit.get_all_block_population() / auxiliary_data_max_population_of_circuits
    ) * IMPORTANCE_ALPHA + circuit.industrialization * (1 - IMPORTANCE_ALPHA)


def get_block_importance(block: Block) -> float:
    return (
        block.citizens.amount / auxiliary_data_max_population_of_circuits
    ) * IMPORTANCE_ALPHA + block.industrialization * (1 - IMPORTANCE_ALPHA)


def set_importance(ci: list[Circuit]):
    for circuit in ci:
        for block in circuit.blocks:
            block.importance = get_block_importance(block)
        circuit.importance = get_circuit_importance(circuit)

set_importance(electric_system_circuits_list)

max_population_data = {
    "Max Population of Circuits": [auxiliary_data_max_population_of_circuits],
    "Max Population of Blocks": [auxiliary_data_max_population_of_block],
}

max_population_df = pd.DataFrame(max_population_data)
# TODO : update plotly
max_population_df

,Max Population of Circuits,Max Population of Blocks
0,19474,4993


In [22]:
# All data of one block of a circuit (importance, citizens, industrialization, demand per hour)

circuit_stats = electric_system_circuits_list[0]

blocks = circuit_stats.blocks

citizens = []
importance = []
demand_per_hour = []

for block in blocks:
    citizens.append(block.citizens.amount)
    importance.append(block.importance)
    demand_per_hour.append(sum(block.predicted_demand_per_hour))

block_stats_df = pd.DataFrame(
    {
        "Citizens": citizens,
        "Importance": importance,
        "Demand per Hour": demand_per_hour,
    }
)

fig_citizens = px.bar(
    block_stats_df,
    x=block_stats_df.index,
    y="Citizens",
    title=f"Block Citizens in Circuit {circuit_stats.id}",
    height=800,
)

fig_importance = px.bar(
    block_stats_df,
    x=block_stats_df.index,
    y="Importance",
    title=f"Block Importance in Circuit {circuit_stats.id}",
    height=800,
)

fig_demand_per_hour = px.bar(
    block_stats_df,
    x=block_stats_df.index,
    y="Demand per Hour",
    title=f"Block Demand per Hour in Circuit {circuit_stats.id}",
    height=800,
)


fig_citizens.show()
fig_importance.show()
fig_demand_per_hour.show()

In [23]:
# Compare data from circuits

citizens = []
importance = []
demand_per_hour = []

for circuit in sorted_electric_system_circuit_list:
    
    importance.append(circuit.importance)
    demand_per_hour.append(
        sum([sum(block.predicted_demand_per_hour) for block in circuit.blocks])
    )

# Separate for properties
citizens = [
    sum(block.citizens.amount for block in circuit.blocks)
    for circuit in electric_system_circuits_list
]

circuit_comparison_df = pd.DataFrame(
    {   "Circuit ID": [c.id for c in sorted_electric_system_circuit_list], 
        "Citizens": citizens,
        "Importance": importance,
        "Demand per Hour": demand_per_hour,
    }
)

fig_citizens = px.bar(
    circuit_comparison_df,
    x="Circuit ID",
    y="Citizens",
    title="Total Citizens per Circuit",
    height=800,
    color=colors,
    color_discrete_sequence=px.colors.qualitative.Plotly,
    labels={"color": "Thermoelectric ID"},
)

fig_importance = px.bar(
    circuit_comparison_df,
    x="Circuit ID",
    y="Importance",
    title="Importance per Circuit",
    height=800,
    color=colors,
    color_discrete_sequence=px.colors.qualitative.Plotly,
    labels={"color": "Thermoelectric ID"},
)

fig_demand_per_hour = px.bar(
    circuit_comparison_df,
    x="Circuit ID",
    y="Demand per Hour",
    title="Total Demand per Circuit in a Day",
    height=800,
    color=colors,
    color_discrete_sequence=px.colors.qualitative.Plotly,
    labels={"color": "Thermoelectric ID"},
)

fig_citizens.show()
fig_importance.show()
fig_demand_per_hour.show()

In [24]:
circuit_id = "C0"
circuit = next(c for c in electric_system_circuits_list if c.id == circuit_id)

block_demand_data = {"Block ID": [], "Hour": [], "Demand": []}

for i, block in enumerate(circuit.blocks):
    for hour, demand in enumerate(block.predicted_demand_per_hour):
        block_demand_data["Block ID"].append(f"Block {i + 1}")
        block_demand_data["Hour"].append(hour)
        block_demand_data["Demand"].append(demand)

block_demand_df = pd.DataFrame(block_demand_data)

fig_block_demand = px.line(
    block_demand_df,
    x="Hour",
    y="Demand",
    color="Block ID",
    title=f"Hourly Demand per Block in Circuit {circuit_id}",
    labels={"Demand": "Demand (kWh)", "Hour": "Hour"},
)

fig_block_demand.show()

circuit_demand_data = {
    "Hour": list(range(24)),
    "Demand": [
        sum(block.predicted_demand_per_hour[hour] for block in circuit.blocks)
        for hour in range(24)
    ],
}

circuit_demand_df = pd.DataFrame(circuit_demand_data)

fig_circuit_demand = px.line(
    circuit_demand_df,
    x="Hour",
    y="Demand",
    title=f"Hourly Demand of Circuit {circuit_id}",
    labels={"Demand": "Demand (kWh)", "Hour": "Hour"},
)

fig_circuit_demand.show()

# Simulation

In [25]:
from src.people import ThermoelectricAgent, ThermoelectricAgentPerception
from src.worldstate import WorldState


class Simulation:
    def __init__(
        self,
        circuits: list[Circuit],
        thermoelectrics: list[Thermoelectric],
        distance_matrix: list[list[float]],
        get_circuit_importance: callable,
        get_block_importance: callable,
    ):
        self.world_state_manager = WorldState(
            circuits=circuits,
            thermoelectrics=thermoelectrics,
            distance_matrix=distance_matrix,
            get_block_importance=get_block_importance,
            get_circuit_importance=get_circuit_importance,
        )

        self.thermoelectric_agents: list[ThermoelectricAgent] = []

    def create_thermoelectric_agent_perception(
        self, thermoelectric
    ) -> ThermoelectricAgentPerception:

        return ThermoelectricAgentPerception(
            thermoelectric=thermoelectric,
            general_deficit=self.world_state_manager.general_deficit,
            general_demand=self.world_state_manager.general_demand,
            general_offer=self.world_state_manager.general_offer,
        )

    def simulate(self, simulation_days=50):

        for thermoelectric in self.world_state_manager.thermoelectrics:
            agent_initial_perception = self.create_thermoelectric_agent_perception(thermoelectric=thermoelectric)
            print("agent perception")
            print(agent_initial_perception)
            thermoelectric_agent = ThermoelectricAgent(
                name=f"Agent of {thermoelectric.id}",
                thermoelectric=thermoelectric,
                perception=agent_initial_perception,
            )

            self.thermoelectric_agents.append(thermoelectric_agent)

    # check progress

    # for agent in self.thermoelectric_agents:
    #     print(agent)


simulation = Simulation(
    circuits=electric_system_circuits_list,
    thermoelectrics=electric_system_thermoelectrics_list,
    distance_matrix=matrix,
    get_circuit_importance=get_circuit_importance,
    get_block_importance=get_block_importance,
)

simulation.simulate()

agent perception
[<src.part.Boiler object at 0x71f18bc510f0>, <src.part.Boiler object at 0x71f18bc516f0>, <src.part.Generator object at 0x71f18bc50340>, <src.part.Coils object at 0x71f18bc50490>, <src.part.SteamTurbine object at 0x71f18bc50610>]
[(<src.part.Boiler object at 0x71f18bc510f0>, 4870.986561332986), (<src.part.Boiler object at 0x71f18bc516f0>, 4870.986561332986), (<src.part.Generator object at 0x71f18bc50340>, 0), (<src.part.Coils object at 0x71f18bc50490>, 0), (<src.part.SteamTurbine object at 0x71f18bc50610>, 0)]
{
            "thermoelectric: {'id': 'Th0', 'total_capacity': 9741.973122665971, 'current_capacity': 0, 'stored_energy': 0, 'parts': [{'type': 'Boiler', 'is_working': True, 'estimated_remaining_life': 14.856244220034869}, {'type': 'Boiler', 'is_working': True, 'estimated_remaining_life': 57.666486693912624}, {'type': 'Generator', 'is_working': True, 'estimated_remaining_life': 10.413582103856283}, {'type': 'Coils', 'is_working': True, 'estimated_remaining_life': 